In [1]:
'''Importing the dependencies and libraries'''
import sys, dlib, random, os, time
from tqdm.notebook import tqdm
from IPython.display import display


# Analytics, ML and Visualization libraries
# import modin.pandas as pd
import pandas as pd
from PIL import Image
import cv2
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt 
import matplotlib.image as img_plt 
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Deep learning libraries
import torch
from torch import nn
from torch.nn import functional as F
from utils.download import download_url_to_file
from torch.utils.data import Dataset
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

print(sys.executable)

C:\Users\rakes\anaconda3\envs\productTemplate\python.exe


# Preprocessing

1. Provide the required values for the variables (face_detector, img_path, data_fol, im_size) that are assinged!
2. Provide the classes folder name in numeric values like (0,1,2,...)
3. Provide the faces images in respected classes folder.
Data Folder Structure:
        --> data:
               --> class 1:
                       --> Image 1
                       --> Image 2
                       --> Image 3
                           .
                           .
                           .
               --> class 2:
                       --> Image 1
                       --> Image 2
                       --> Image 3
                           .
                           .
                           ....

                   .
                   .
                   ....
4. Provide face detector 'dlib' or None. Default cv2 cascade classifier is selected.
5. Provide the  im_size depends of model input shape [224*224] is selected currently.  

In [3]:
# Assigin "dlib" to run dib face detector
# Assign "None" to run cv2 Cascade Classifier 
face_detector="dlib" 
img_path= r"path\0\Claudia_Pechstein_0003.jpg" # image path 
data_fol=r"path" # provide the data directory path
im_size= 224#image size

# Default 'cv2.CascadeClassifier' is selected 
if face_detector=="dlib":
    detector = dlib.get_frontal_face_detector()
else:    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

In [4]:
# Select True if you want to view the image
if False:
    img = cv2.imread(img_path) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    if face_detector=="dlib":
        faces = detector(gray)
        print(faces)
        for face in faces:
            x, y, w, h = (face.left(), face.top(), face.width(), face.height())
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2) 
    else:
        faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
        for (x,y,w,h) in faces: 
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2) 
            roi_gray = gray[y:y+h, x:x+w] 
            roi_color = img[y:y+h, x:x+w]
                
    cv2.imshow('img',img) 
    cv2.waitKey(10)
    cv2.destroyAllWindows() 

To prepare "data.csv" datafame with columns "Image Path", "Face", "Bounding Box", "Aspect ratio", "Target" for training purpose with pandas library.

In [1]:
classes=os.listdir(data_fol)
rows_list=[]
for i in tqdm(classes):
    fol=data_fol+f"\{i}"
    imgs=os.listdir(fol)
    for j in imgs:
        img = cv2.imread(fol+f"\{j}") 
        img=cv2.resize(img, (im_size, im_size), interpolation = cv2.INTER_LINEAR)
        H, W, c = img.shape
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        if face_detector=="dlib":
            faces = detector(gray)
            cls_n=None
            if faces:
                face=faces[0]
                x, y, w, h = (face.left(), face.top(), face.width(), face.height());bb=abs(x),abs(y),abs(x+w),abs(y+h)
            else:
                bb= (0,0,W,H)
                cls_n=len(classes)+1   
            rows_list.append({'Image Path':fol+f"\{j}", 'Face': 1 if faces else 0, 'Bounding Box':bb ,'Aspect ratio':(w,h),'Target':cls_n if cls_n else i})
        else:
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            cls_n=None
            if type(faces)!=tuple and faces.any():
                x,y,w,h=faces[0];bb=abs(x),abs(y),abs(x+w),abs(y+h)
            else:
                bb= (0,0,W,H)
                cls_n=len(classes)+1
            rows_list.append({'Image Path':fol+f"\{j}", 'Face': 1 if type(faces)!=tuple and faces.any() else 0, 'Bounding Box':bb ,'Aspect ratio':(w,h),'Target':cls_n if cls_n else i})
df = pd.DataFrame(rows_list) 
df.astype(object)
display(df.head())   
df.to_csv("data.csv",index=False)
print("================= 'data.csv' Generated =================")

In [6]:
df.info() # provide the details of dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7606 entries, 0 to 7605
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Image Path    7606 non-null   object
 1   Face          7606 non-null   int64 
 2   Bounding Box  7606 non-null   object
 3   Aspect ratio  7606 non-null   object
 4   Target        7606 non-null   object
dtypes: int64(1), object(4)
memory usage: 297.2+ KB


kmeans cluster and k nearest neighbor algorithim is used to find anchor and to prepare the dataset.

In [2]:
#Clustering Algorithm
def kmeans_anchors(boxes, k):
    # Normalize boxes
    boxes = np.array(boxes)
    kmeans = KMeans(n_clusters=k, random_state=0).fit(boxes)
    anchors = kmeans.cluster_centers_
    return anchors

face_df=df[df["Face"]==1]
print(face_df.shape)
anchor_=kmeans_anchors(face_df["Aspect ratio"].tolist(),1)
anchor_=np.array(anchor_[0]).astype(int)
face_df["Anchor"]=[list(anchor_)]*face_df.shape[0]

''' To find cluster with respective to classes'''
# anchor_dic={}
# for un_ in face_df["Target"].unique().tolist():
#     tempdf=face_df[face_df["Target"]==un_]
#     anchor_dic.update({un_:kmeans_anchors(tempdf["Aspect ratio"].tolist(), 1)[0]})
# print(anchor_dic)  

display(face_df)

In [3]:
anchor_im={} #anchor image
anchor_bb={} #anchor bounding box

for un_ in tqdm(face_df["Target"].unique().tolist()):
    tempdf=face_df[face_df["Target"]==un_]
    points=[tuple(tempdf["Anchor"].tolist()[0])]+tempdf["Aspect ratio"].tolist()
    nbrs = NearestNeighbors(n_neighbors=2).fit(points)
    distances, indices = nbrs.kneighbors(points)
    if indices.any():
        anchor_im.update({un_:tempdf.iloc[indices[0][1]-1]["Image Path"]})
        anchor_bb.update({un_:tempdf.iloc[indices[0][1]-1]["Bounding Box"]})

def anchor_image(row):
    global anchor_im
    return anchor_im[row["Target"]]

def anchor_boundingbox(row):
    global anchor_bb
    return anchor_bb[row["Target"]]    
    
    

face_df['Anchor Image'] = face_df.apply(anchor_image, axis=1)
face_df['Anchor bb'] = face_df.apply(anchor_boundingbox, axis=1)

display(face_df)


To prepare final "face_data.csv" datafame with columns "Image Path", "Face", "Bounding Box", "Aspect ratio", "Target", "Anchor", "Anchor Image", "Anchor bb", "Negative Image", "Negative bb" for training.

In [4]:
neg_ls=[] #negative image path
neg_bb=[] #negative face bounding box

for row in tqdm(face_df.iterrows()):
    ex_id=row[1]["Target"]
    tempdf=face_df[face_df["Target"]!=ex_id]
    rand_path=random.choice(tempdf["Image Path"].tolist())
    rand_bb=tempdf[tempdf["Image Path"]==rand_path]["Bounding Box"].tolist()[0]
    neg_ls.append(rand_path)
    neg_bb.append(rand_bb)

face_df["Negative Image"]=neg_ls
face_df["Negative bb"]=neg_bb
display(face_df.head())
face_df.to_csv("face_data.csv",index=False)
print("++++++++++++++++++++++++ 'face_data.csv' is Generated ++++++++++++++++++++++++")

Visualising and analysing the non detected face images by face detector.

In [5]:
display(df[df["Face"]==0]); len(df[df["Face"]==0]) # Filtering the non-detected face

In [6]:
# Visualising  the non- detected face and anlysis
for row in df[df["Face"]==0].iterrows():
    print(row[1]["Image Path"])
    im = img_plt.imread(row[1]["Image Path"]) 
    plt.imshow(im) 
    plt.show()

In [12]:
'''
dlib face detector detected around 99% of face with provided dataset
cv2 cascade classifier detected around 95% of face with provided dataset
'''

print("Percentage of face detected by dlib face detector in provided dataset",(df["Face"].sum()/df.shape[0])*100) # cv2 face cascade 95%

Percentage of face detected by dlib face detector in provided dataset 99.35577175913753


# Training

# FaceNet Model

FaceNet is the name of the facial recognition system that was proposed by Google Researchers in 2015 in the paper titled FaceNet: A Unified Embedding for Face Recognition and Clustering.

Reference:

    -> https://github.com/davidsandberg/facenet
    -> https://github.com/timesler/facenet-pytorch/tree/master
    -> https://medium.com/@mohitsaini_54300/train-facenet-with-triplet-loss-for-real-time-face-recognition-a39e2f4472c3
    -> https://datahacker.rs/025-facenet-a-unified-embedding-for-face-recognition-and-clustering-in-pytorch/
    -> https://www.geeksforgeeks.org/facenet-using-facial-recognition-system/

Architecture:

            Input images [anchor, positive, negative]
            Batch Input [Tensor:torch.Size([1, 3, Image size, Image size])]
                               ⬇️
            FaceNet DNN network (InceptionResnetV1 model)
                    (pretrained vggface weights)
                               ⬇️
                face embbedings [torch.Size([1, 512])]
                               ⬇️
                      Triplet loss function                      
                  

This is a Pytorch implementation of the face recognizer.

# DataLoader

In [13]:
imageSize=224 #input image size

'''
Custom dataloader for Facenet model.
Returns: anchor (Tensor:torch.Size([1, 3, 224, 224])), positive (Tensor:torch.Size([1, 3, 224, 224])), negative (Tensor:torch.Size([1, 3, 224, 224]))
'''
class customDataset(Dataset):
    def __init__(self, csv_path, imageSize,transform=None):
        self.df_data=pd.read_csv(csv_path)  
        self.transform = transform
        self.positiveImages = self.df_data["Image Path"].tolist()
        self.negativeImages = self.df_data["Negative Image"].tolist()
        self.anchorImages = self.df_data["Anchor Image"].tolist()
        self.pos_bb = self.df_data["Bounding Box"].tolist()
        self.neg_bb=self.df_data["Negative bb"].tolist()
        self.anchor_bb=self.df_data["Anchor bb"].tolist()
        self.im_size=imageSize
    
    
    def read_image(self,path,bb):
        try:
            image=cv2.imread(path)
            image=cv2.resize(image, (self.im_size, self.im_size), interpolation = cv2.INTER_LINEAR)
            image=image[bb[1]:bb[3],bb[0]:bb[2]]
            image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image=Image.fromarray(image)
            return image 
        except Exception as e:
            print("============================================")
            print(e,path,bb)
        
    def __len__(self):
        return len(self.positiveImages)

    def __getitem__(self, idx):
        pos = self.read_image(self.positiveImages[idx],eval(self.pos_bb[idx]))
        neg= self.read_image(self.negativeImages[idx],eval(self.neg_bb[idx]))
        anchor= self.read_image(self.anchorImages[idx],eval(self.anchor_bb[idx]))
        
        
        if self.transform:
            positive = self.transform(pos)
            negative = self.transform(neg)
            anchor = self.transform(anchor)
        return anchor, positive, negative

# Define transformations for the data
transform = transforms.Compose([
    transforms.Resize((imageSize, imageSize)),  # Resize the image to the desired size
    transforms.ToTensor()
])

# Create the dataset and dataloader
dataset = customDataset('./face_data.csv', imageSize, transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

# Example of iterating through the dataloader
for anchor, positive, negative in tqdm(dataloader):
    print(anchor.size(), positive.size(),negative.size()) #size of the tensor
    break


  0%|          | 0/945 [00:00<?, ?it/s]

torch.Size([8, 3, 224, 224]) torch.Size([8, 3, 224, 224]) torch.Size([8, 3, 224, 224])


# Model arhitecture

In [2]:
def get_torch_home():
    torch_home = os.path.expanduser(
        os.getenv(
            'TORCH_HOME',
            os.path.join(os.getenv('XDG_CACHE_HOME', '~/.cache'), 'torch')
        )
    )
    return torch_home

def load_weights(mdl):
    path = 'https://github.com/timesler/facenet-pytorch/releases/download/v2.2.9/20180402-114759-vggface2.pt' #'vggface2' weights
    
    model_dir = os.path.join(get_torch_home(), 'checkpoints')
    os.makedirs(model_dir, exist_ok=True)

    cached_file = os.path.join(model_dir, os.path.basename(path))
    if not os.path.exists(cached_file):
        download_url_to_file(path, cached_file)

    state_dict = torch.load(cached_file)
    mdl.load_state_dict(state_dict)

class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super().__init__()
        self.conv = nn.Conv2d(
            in_planes, out_planes,
            kernel_size=kernel_size, stride=stride,
            padding=padding, bias=False
        ) # verify bias false
        self.bn = nn.BatchNorm2d(
            out_planes,
            eps=0.001, # value found in tensorflow
            momentum=0.1, # default pytorch value
            affine=True
        )
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class Block35(nn.Module):

    def __init__(self, scale=1.0):
        super().__init__()

        self.scale = scale

        self.branch0 = BasicConv2d(256, 32, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(256, 32, kernel_size=1, stride=1),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(256, 32, kernel_size=1, stride=1),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1)
        )

        self.conv2d = nn.Conv2d(96, 256, kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        out = torch.cat((x0, x1, x2), 1)
        out = self.conv2d(out)
        out = out * self.scale + x
        out = self.relu(out)
        return out


class Block17(nn.Module):

    def __init__(self, scale=1.0):
        super().__init__()

        self.scale = scale

        self.branch0 = BasicConv2d(896, 128, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(896, 128, kernel_size=1, stride=1),
            BasicConv2d(128, 128, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(128, 128, kernel_size=(7,1), stride=1, padding=(3,0))
        )

        self.conv2d = nn.Conv2d(256, 896, kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        out = torch.cat((x0, x1), 1)
        out = self.conv2d(out)
        out = out * self.scale + x
        out = self.relu(out)
        return out


class Block8(nn.Module):

    def __init__(self, scale=1.0, noReLU=False):
        super().__init__()

        self.scale = scale
        self.noReLU = noReLU

        self.branch0 = BasicConv2d(1792, 192, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(1792, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 192, kernel_size=(1,3), stride=1, padding=(0,1)),
            BasicConv2d(192, 192, kernel_size=(3,1), stride=1, padding=(1,0))
        )

        self.conv2d = nn.Conv2d(384, 1792, kernel_size=1, stride=1)
        if not self.noReLU:
            self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        out = torch.cat((x0, x1), 1)
        out = self.conv2d(out)
        out = out * self.scale + x
        if not self.noReLU:
            out = self.relu(out)
        return out


class Mixed_6a(nn.Module):

    def __init__(self):
        super().__init__()

        self.branch0 = BasicConv2d(256, 384, kernel_size=3, stride=2)

        self.branch1 = nn.Sequential(
            BasicConv2d(256, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 192, kernel_size=3, stride=1, padding=1),
            BasicConv2d(192, 256, kernel_size=3, stride=2)
        )

        self.branch2 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        out = torch.cat((x0, x1, x2), 1)
        return out


class Mixed_7a(nn.Module):

    def __init__(self):
        super().__init__()

        self.branch0 = nn.Sequential(
            BasicConv2d(896, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 384, kernel_size=3, stride=2)
        )

        self.branch1 = nn.Sequential(
            BasicConv2d(896, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 256, kernel_size=3, stride=2)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(896, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 256, kernel_size=3, stride=1, padding=1),
            BasicConv2d(256, 256, kernel_size=3, stride=2)
        )

        self.branch3 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class InceptionResnetV1(nn.Module):
    def __init__(self, pretrained="vggface2", classify=False, num_classes=None, dropout_prob=0.6):
        super().__init__()

        self.pretrained = pretrained
        self.classify = classify
        self.num_classes = num_classes
        tmp_classes = 8631  #'vggface2' output layer

        # Define layers
        self.conv2d_1a = BasicConv2d(3, 32, kernel_size=3, stride=2)
        self.conv2d_2a = BasicConv2d(32, 32, kernel_size=3, stride=1)
        self.conv2d_2b = BasicConv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.maxpool_3a = nn.MaxPool2d(3, stride=2)
        self.conv2d_3b = BasicConv2d(64, 80, kernel_size=1, stride=1)
        self.conv2d_4a = BasicConv2d(80, 192, kernel_size=3, stride=1)
        self.conv2d_4b = BasicConv2d(192, 256, kernel_size=3, stride=2)
        self.repeat_1 = nn.Sequential(
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
        )
        self.mixed_6a = Mixed_6a()
        self.repeat_2 = nn.Sequential(
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
        )
        self.mixed_7a = Mixed_7a()
        self.repeat_3 = nn.Sequential(
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
        )
        self.block8 = Block8(noReLU=True)
        self.avgpool_1a = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_prob)
        self.last_linear = nn.Linear(1792, 512, bias=False)
        self.last_bn = nn.BatchNorm1d(512, eps=0.001, momentum=0.1, affine=True)

        if pretrained is not None:
            self.logits = nn.Linear(512, tmp_classes)
            load_weights(self)

        if self.classify and self.num_classes is not None:
            self.logits = nn.Linear(512, self.num_classes)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.device = torch.device(device)
        self.to(device)

    def forward(self, x):
        """Calculate embeddings or logits given a batch of input image tensors.

        Arguments:
            x {torch.tensor} -- Batch of image tensors representing faces.

        Returns:
            torch.tensor -- Batch of embedding vectors or multinomial logits.
        """
        x = self.conv2d_1a(x)
        x = self.conv2d_2a(x)
        x = self.conv2d_2b(x)
        x = self.maxpool_3a(x)
        x = self.conv2d_3b(x)
        x = self.conv2d_4a(x)
        x = self.conv2d_4b(x)
        x = self.repeat_1(x)
        x = self.mixed_6a(x)
        x = self.repeat_2(x)
        x = self.mixed_7a(x)
        x = self.repeat_3(x)
        x = self.block8(x)
        x = self.avgpool_1a(x)
        x = self.dropout(x)
        x = self.last_linear(x.view(x.shape[0], -1))
        x = self.last_bn(x)
        if self.classify:
            x = self.logits(x)
        else:
            x = F.normalize(x, p=2, dim=1)
        return x


In [15]:
#Loss Function
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = F.pairwise_distance(anchor, positive, p=2)
        distance_negative = F.pairwise_distance(anchor, negative, p=2)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

In [16]:
# saving checkpoint for every epochs
def save_checkpoint(model, optimizer, epoch, loss, path):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss
    }
    torch.save(checkpoint, path)
    
# loading checkpoint 
def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model, optimizer, epoch, loss

In [17]:
# Training Epochs
num_epochs = 100 #number of epochs

checkpoint_fol = './model/' # provide directory of model to be saved
checkpoint_path=None # provide the path to load the trained model

model = InceptionResnetV1() #intialze the model
triplet_loss = TripletLoss(margin=1.0) #intialze the loss function
optimizer = optim.Adam(model.parameters(), lr=0.001) #adam optimizer
start_epoch=0
# If 'checkpoint_path' not None and path is provided. The model will be loaded and weights will be intialized with trained weights
if checkpoint_path and os.path.exists(checkpoint_path):
    model, optimizer, start_epoch, _ = load_checkpoint(model, optimizer, checkpoint_path)

In [20]:
for epoch in tqdm(range(start_epoch,num_epochs)):
    model.train()
    running_loss = 0.0
    try:
        for batch_idx, (anchor, positive, negative) in tqdm(enumerate(dataloader)):
            anchor, positive, negative = anchor, positive, negative
            
            optimizer.zero_grad()
            
            anchor_embedding = model(anchor)
            positive_embedding = model(positive)
            negative_embedding = model(negative)
            
            loss = triplet_loss(anchor_embedding, positive_embedding, negative_embedding)
            loss.backward()
            optimizer.step()        
            running_loss += loss.item()
    except Exception as e:
        print(e)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')
    file_=f'{epoch+1}_Loss_{running_loss/len(dataloader):.4f}'
    save_checkpoint(model, optimizer, epoch, running_loss, checkpoint_fol+f"{file_}.pt")
print("Finished Training")    

  0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epoch [1/100], Loss: 0.8586


0it [00:00, ?it/s]

Epoch [2/100], Loss: 0.8046


...

Epoch [9/100], Loss: 0.4543


0it [00:00, ?it/s]

Epoch [10/100], Loss: 0.4431


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [19]:
'''To save the checkpoint for trained weights'''
# print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')
# file_=f'{epoch+1}_Loss_{running_loss/len(dataloader):.4f}'
# save_checkpoint(model, optimizer, epoch, running_loss, checkpoint_fol+f"{file_}.pt")

Epoch [1/100], Loss: 0.8900


# Testing

In [7]:
class Face_recognition():
    im_size=224
    
    transform = transforms.Compose([
    transforms.Resize((im_size, im_size)),  # Resize the image to the desired size
    transforms.ToTensor()]) 

    master_embedding={}
    
    def __init__(self,model, optimizer,trained_data_df,checkpoint=None):
        master_df=pd.read_csv(trained_data_df,usecols=["Anchor Image","Anchor bb","Target"])
        master_df=master_df.groupby("Anchor Image").first()
        self.model=model
        if checkpoint:self.model=self.load_checkpoint(self.model, optimizer, checkpoint)
        self.model.eval()    
        for i in tqdm(master_df.iterrows()):
            detect_img=self.read_image(i[0],eval(i[1]["Anchor bb"]))
            with torch.no_grad():
                face_embedding = self.model(detect_img.unsqueeze(0))
            Face_recognition.master_embedding.update({i[1]["Target"]:face_embedding})    

    def load_checkpoint(self,model, optimizer, path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print("loaded")
        return model
    
    
    def read_image(self,path,bb):
        image=cv2.imread(path)
        image=cv2.resize(image, (self.im_size, self.im_size), interpolation = cv2.INTER_LINEAR)
        image=image[bb[1]:bb[3],bb[0]:bb[2]]
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image=Image.fromarray(image)
        return Face_recognition.transform(image)    

    def preprocess(self,image):
        im_size= Face_recognition.im_size
        image=cv2.resize(image, (im_size, im_size), interpolation = cv2.INTER_LINEAR)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image=Image.fromarray(image)
        return image 

    def compare_embeddings(self,embedding1, embedding2, similarity_=False,threshold=0.6):
        if similarity_:
            cos = nn.CosineSimilarity(dim=1)
            output = cos(embedding1, embedding2)
            return float(output), bool(output> threshold)  
        else:
            distance = np.linalg.norm(embedding1.detach().numpy() - embedding2.detach().numpy())
            return distance, distance < threshold
    
    def detect(self,img, similarity_=False,thresh=0.6):
        detect_img=self.preprocess(img)
        detect_img = Face_recognition.transform(detect_img)
        with torch.no_grad():
            face_embedding = self.model(detect_img.unsqueeze(0))

        predictions=[]
        for class_name, fea in Face_recognition.master_embedding.items():
            val, flag =self.compare_embeddings(face_embedding, fea, similarity_,thresh)
            predictions.append([class_name,val, flag])

        predictions=np.array(predictions)
        if predictions.any():
            predictions=predictions[np.where(predictions[:,2]==True)]
            if predictions.any():
                if similarity_:
                    predicted_class=predictions[predictions[:, 1].argsort()][-1][0]
                    val=predictions[predictions[:, 1].argsort()][-1][1]
                else:
                    predicted_class=predictions[predictions[:, 1].argsort()][0][0]
                    val=predictions[predictions[:, 1].argsort()][0][1] 
                return {predicted_class:val}, predictions[predictions[:, 1].argsort()]
        return {}       

In [8]:
img_path=r"path\6\Mark_Richt_0002.jpg"
face_detector="dlib"
im_size=224 #image size
check_point_path="./model 2/8_Loss_0.4777.pt"
master_data="./face_data.csv"

model = InceptionResnetV1()
opt=optim.Adam(model.parameters(), lr=0.001)
fr=Face_recognition(model,opt,master_data,check_point_path)

loaded


0it [00:00, ?it/s]

In [9]:
if face_detector=="dlib":
    detector = dlib.get_frontal_face_detector()
else:    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
    
if True:
    
    start_time=time.time()
    img = cv2.imread(img_path) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    crop_img=[]
    if face_detector=="dlib":
        faces = detector(gray)
        for face in faces:
            x, y, w, h = (face.left(), face.top(), face.width(), face.height())
            # cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
            crop_img.append(img[y:y+h,x:x+w])
    else:
        faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
        for (x,y,w,h) in faces: 
            # cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2) 
            roi_gray = gray[y:y+h, x:x+w] 
            roi_color = img[y:y+h, x:x+w]
            crop_img.append(roi_color)

    results={}
    
    if crop_img:
        for c_img in crop_img:
            result,__=fr.detect(c_img, 0.6)
            results.update(result)
            
    img=cv2.resize(img, (420,640), interpolation = cv2.INTER_LINEAR)    
    img = cv2.putText(img, f"Detected class: {results.keys() if results else 'Nil' }", (5, 10), cv2.FONT_HERSHEY_SIMPLEX,  0.5, (255, 0, 0) , 1, cv2.LINE_AA)       
    print("Time taken: ",time.time()-start_time) 
    print("Predition: ", results)
    cv2.imshow('img',img) 
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

Time taken:  0.4645073413848877
Predition:  {317.0: 0.9998830556869507}


In [24]:
# Checking the prediction values
np.set_printoptions(suppress=True, precision=4)
__

array([[362.    ,   0.6016,   1.    ],
       [341.    ,   0.6022,   1.    ],
       [307.    ,   0.6076,   1.    ],
       [841.    ,   0.6085,   1.    ],
       [452.    ,   0.6092,   1.    ],
       [592.    ,   0.6112,   1.    ],
       [832.    ,   0.6119,   1.    ],
       [626.    ,   0.6159,   1.    ],
       [474.    ,   0.6161,   1.    ],
       [194.    ,   0.62  ,   1.    ],
       [481.    ,   0.6216,   1.    ],
       [760.    ,   0.6223,   1.    ],
       [378.    ,   0.6268,   1.    ],
       [601.    ,   0.6312,   1.    ],
       [805.    ,   0.6315,   1.    ],
       [723.    ,   0.632 ,   1.    ],
       [479.    ,   0.6323,   1.    ],
       [614.    ,   0.6328,   1.    ],
       [375.    ,   0.635 ,   1.    ],
       [874.    ,   0.6373,   1.    ],
       [541.    ,   0.6384,   1.    ],
       [300.    ,   0.6387,   1.    ],
       [696.    ,   0.642 ,   1.    ],
       [241.    ,   0.648 ,   1.    ],
       [488.    ,   0.6487,   1.    ],
       [900.    ,   0.649

# Evaluation:

In [7]:
eval_df= pd.read_csv("face_data.csv", usecols=["Image Path","Face","Bounding Box","Target"]) 
eval_df=eval_df[eval_df["Face"]==1]
# display(eval_df.head())

results=[]
for i in tqdm(eval_df.iterrows()):
    img=cv2.imread(i[1]["Image Path"])
    img=cv2.resize(img, (224, 224), interpolation = cv2.INTER_LINEAR)
    bb=eval(i[1]["Bounding Box"])
    crp_img=img[bb[1]:bb[3],bb[0]:bb[2]]
    result,__=fr.detect(crp_img, 0.5)
    if result:results.append(int(list(result.keys())[0]))
    else:results.append(None)  
eval_df["Predicted"]=results 
display(eval_df)
    


In [26]:
from sklearn.metrics import confusion_matrix, classification_report
y_true = eval_df["Target"].tolist()
y_pred = eval_df["Predicted"].tolist()
print("============== Confusion Matrix===================")
confusion_matrix(y_true, y_pred)

============== Confusion Matrix===================


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [27]:
tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
print(f"True negative:{tn}, True positive: {tp}, False negative:{fn}, False positive:{fp}")

True negative:0, True positive: 1, False negative:1, False positive:2


In [28]:
print("+++++++++++Classification Report++++++++++++++++++++++++")
print(classification_report(y_true, y_pred))

+++++++++++Classification Report++++++++++++++++++++++++
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         9
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         9
          11       0.00      0.00      0.00         7
          12       0.00      0.00      0.00         6
          13       0.00      0.00      0.00         9
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         4
          16       0.00 

C:\Users\rakes\anaconda3\envs\productTemplate\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rakes\anaconda3\envs\productTemplate\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rakes\anaconda3\envs\productTemplate\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz